<a href="https://colab.research.google.com/github/Josep-at-work/curso-series-temporales/blob/master/2.%20Serie%20temporal%20en%20R%20y%20Python/Python/2_Python_Objeto_serie_temporal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Importing the Packages

In [4]:
import pandas as pd
import numpy as np

### Importing the data 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [24]:
# raw_csv_data = pd.read_csv("Index2018.csv") 
raw_csv_data = pd.read_csv("/content/drive/MyDrive/Udemy/Series Temporales/Data/telemetry.csv")
df_comp = raw_csv_data.copy() 

### Lenght of the Time Period

In [25]:
df_comp.describe() #only stats for numeric variables

,machineID,volt,rotate,pressure,vibration
count,876100.000000,876100.000000,876100.000000,876100.000000,876100.000000
mean,50.500000,170.777736,446.605119,100.858668,40.385007
std,28.866087,15.509114,52.673886,11.048679,5.370361
min,1.000000,97.333604,138.432075,51.237106,14.877054
25%,25.750000,160.304927,412.305714,93.498181,36.777299
50%,50.500000,170.607338,447.558150,100.425559,40.237247
75%,75.250000,181.004493,482.176600,107.555231,43.784938
max,100.000000,255.124717,695.020984,185.951998,76.791072


In [8]:
df_comp.head()

,datetime,machineID,volt,rotate,pressure,vibration
0,1/1/2015 6:00:00 AM,1,176.217853,418.504078,113.077935,45.087686
1,1/1/2015 7:00:00 AM,1,162.879223,402.747490,95.460525,43.413973
2,1/1/2015 8:00:00 AM,1,170.989902,527.349825,75.237905,34.178847
3,1/1/2015 9:00:00 AM,1,162.462833,346.149335,109.248561,41.122144
4,1/1/2015 10:00:00 AM,1,157.610021,435.376873,111.886648,25.990511


In [9]:
df_comp.datetime.describe() 

count                     876100
unique                      8761
top       10/27/2015 11:00:00 PM
freq                         100
Name: datetime, dtype: object

### From Text to Date

In [26]:
df_comp.datetime = pd.to_datetime(df_comp.datetime, dayfirst = True)

In [12]:
df_comp.head()

,datetime,machineID,volt,rotate,pressure,vibration
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511


In [15]:
df_comp.datetime.describe(datetime_is_numeric=True)

count                           876100
mean     2015-07-02 17:59:59.999987968
min                2015-01-01 06:00:00
25%                2015-04-02 12:00:00
50%                2015-07-02 18:00:00
75%                2015-10-02 00:00:00
max                2016-01-01 06:00:00
Name: datetime, dtype: object

Los datos registrados completan un año del 1 de enero del 2015 al 2016.

### Setting the Index

This dataset contains values for 100 machines registered at the same time. Thus, in order not to duplicate indexes, I've selected the values for the first machine.

In [27]:
m1 = df_comp[df_comp.machineID == 1]
m1.set_index("datetime", inplace=True)

In [28]:
m1.head()

,machineID,volt,rotate,pressure,vibration
datetime,,,,,
2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686
2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973
2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847
2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144
2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511


In [ ]:
#df_comp.date.describe() #Expect to get an error message because we no longer have a "date" column since it is the index

### Setting the Desired Frequency

asfreq adds the missing datetime values in order to have a persistent frequency. If a new value is added, the values of the feature will be NaN.

In this case the sensor registered for every hour, so no problem.

In [32]:
m1 = m1.asfreq('h') # h(hourly), w(weekly), d(daily), m(monthly), a(anual), b(business dates)

# m1[m1.index.duplicated()] #Now, there are not duplicated indexes!

In [33]:
m1.head()

,machineID,volt,rotate,pressure,vibration
datetime,,,,,
2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686
2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973
2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847
2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144
2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511


In [34]:
m1.asfreq("b") #showing how woul it be to just consider business dates (Mond.-Frid.)

,machineID,volt,rotate,pressure,vibration
datetime,,,,,
2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686
2015-01-02 06:00:00,1,165.967847,504.900773,96.208954,36.841560
2015-01-05 06:00:00,1,158.224742,449.473860,77.044637,37.576166
2015-01-06 06:00:00,1,177.249603,504.803836,93.350302,33.242495
2015-01-07 06:00:00,1,162.040193,423.996563,101.587536,35.669374
...,...,...,...,...,...
2015-12-28 06:00:00,1,180.331045,419.932181,110.422392,29.554339
2015-12-29 06:00:00,1,166.223267,531.455729,85.948154,43.032816
2015-12-30 06:00:00,1,164.551123,511.210126,104.743215,33.491277


### Handling Missing Values

In [35]:
m1.isna()

,machineID,volt,rotate,pressure,vibration
datetime,,,,,
2015-01-01 06:00:00,False,False,False,False,False
2015-01-01 07:00:00,False,False,False,False,False
2015-01-01 08:00:00,False,False,False,False,False
2015-01-01 09:00:00,False,False,False,False,False
2015-01-01 10:00:00,False,False,False,False,False
...,...,...,...,...,...
2016-01-01 02:00:00,False,False,False,False,False
2016-01-01 03:00:00,False,False,False,False,False
2016-01-01 04:00:00,False,False,False,False,False


In [36]:
m1.isna().sum() #No missing numbers, urray!

machineID    0
volt         0
rotate       0
pressure     0
vibration    0
dtype: int64

In [37]:
# df_comp

Using the fillna() function to fill missing values. 
+ Methods:
  + ffill = forward filling, copies the following value
  + bfill = back filling, copies the previous value
+ Value: select a fixed value for all NaN

In [ ]:
# df_comp.spx=df_comp.spx.fillna(method='ffill')

In [38]:
# df_comp.isna().sum()

In [ ]:
# df_comp.ftse=df_comp.ftse.fillna(method='bfill')

In [ ]:
# df_comp.dax=df_comp.dax.fillna(value = df_comp.dax.mean())

In [ ]:
# df_comp.nikkei=df_comp.dax.fillna(method='bfill')

In [39]:
# df_comp.isna().sum()

### Simplifying the Dataset

In order to process faster, is better to only use the feature you are going to analyse each time. 

In the first cell, copying the feature of analysis in a generic variable such as metric, will help simplify and later, repeate the whole process with another variable by just changing the values of the metric feature. 

In [41]:
m1['metric'] = m1.volt

In [42]:
m1.describe()

,machineID,volt,rotate,pressure,vibration,metric
count,8761.0,8761.000000,8761.000000,8761.000000,8761.000000,8761.000000
mean,1.0,170.833898,446.336502,100.668306,40.586309,170.833898
std,0.0,15.314388,52.224261,10.860264,5.542143,15.314388
min,1.0,119.059047,215.619476,58.733724,22.666865,119.059047
25%,1.0,160.382124,412.348209,93.320867,36.847203,160.382124
50%,1.0,170.618459,447.704679,100.319242,40.417442,170.618459
75%,1.0,180.929341,481.358342,107.377475,44.004311,180.929341
max,1.0,237.938518,636.364569,155.257527,67.633435,237.938518


In [43]:
del m1['volt']

In [45]:
m1.describe()

,machineID,rotate,pressure,vibration,metric
count,8761.0,8761.000000,8761.000000,8761.000000,8761.000000
mean,1.0,446.336502,100.668306,40.586309,170.833898
std,0.0,52.224261,10.860264,5.542143,15.314388
min,1.0,215.619476,58.733724,22.666865,119.059047
25%,1.0,412.348209,93.320867,36.847203,160.382124
50%,1.0,447.704679,100.319242,40.417442,170.618459
75%,1.0,481.358342,107.377475,44.004311,180.929341
max,1.0,636.364569,155.257527,67.633435,237.938518


In [49]:
del m1["machineID"]
del m1['rotate']
del m1['pressure']
del m1['vibration']

KeyError: ignored

In [48]:
m1.describe()

,machineID,metric
count,8761.0,8761.000000
mean,1.0,170.833898
std,0.0,15.314388
min,1.0,119.059047
25%,1.0,160.382124
50%,1.0,170.618459
75%,1.0,180.929341
max,1.0,237.938518


### Splitting the Data

Time series data requires of a cronologic order. Thats way we can't shuffle the set of data before the split. Hence, in this case we have to select an specific time where the previous data is the trainign and the test is the rest(future).

In [50]:
size = int(len(m1)*0.8)
size

7008

In [51]:
df = m1.iloc[:size]

In [52]:
df_test = m1.iloc[size:]    

In [53]:
df.tail()

,metric
datetime,
2015-10-20 01:00:00,147.372275
2015-10-20 02:00:00,158.555165
2015-10-20 03:00:00,224.271598
2015-10-20 04:00:00,165.211962
2015-10-20 05:00:00,194.168896


In [54]:
df_test.head()

,metric
datetime,
2015-10-20 06:00:00,149.753524
2015-10-20 07:00:00,191.552953
2015-10-20 08:00:00,159.923512
2015-10-20 09:00:00,153.483418
2015-10-20 10:00:00,158.075270
